<p>
You need to read the python pickled file object which you have prepared in Module-2 Assignment. <br/> Then execute the following operations on that file:
</p>
<p>
Write a Normalization transformer which apply the following normalization on the the  python pickled object you just read
<li>Remove punctuations</li>
<li>Remove stopwords (you can use any standard stopword list or you can create your own)</li>
<li>Print the first 25 documents after applying Lemmatization</li>
<li>Print the first 25 documents after applying Stemming</li>
</p>
<p>
Then Write functions which would execute the following Vectorization on the normalized documents and save the Vectors as a pickle object on your local disk (or your google drive)
<li>Apply TF-IDF vectorization using Gensim</li>
<li>Apply Gensim Doc2Vec technique</li>
</p>

In [0]:
#Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


I am taking a random sample of json files from Cinema and Book Categories

In [0]:
#18 Cinema Files
import os
os.chdir("/content/gdrive/My Drive/Raw_Corpus_Sample/cinema")
os.listdir()

['57ee9f2fdf23b703bff97467.json',
 '58ffc189df23b703bdf5725d.json',
 '58ff81d0df23b703bdf4ed73.json',
 '578fb10ec1808122283046cf.json',
 '588fe381df23b703b96f3a3d.json',
 '594c7b13df23b703bd762af4.json',
 '594c5be2df23b703bd75fa5e.json',
 '57111cbfc1808128a22fb04f.json',
 '5832a052df23b703bf8ca299.json',
 '5733a81cc1808103a3eff478.json',
 '5931bb42df23b703bd4aab59.json',
 '57324feac180817cd239cd98.json',
 '58821e85df23b703b9544023.json',
 '594991dbdf23b703bd7149a9.json',
 '58121789df23b703bf45fb92.json',
 '5710062ec1808128a22d8d6e.json',
 '58830743df23b703b9561bba.json',
 '58878186df23b703b95ea7c5.json']

In [0]:
#14 Book Files
os.chdir("/content/gdrive/My Drive/Raw_Corpus_Sample/books")
os.listdir()

['56d62a52c1808113ffb87f1d.json',
 '56d97be5c1808111c98555ef.json',
 '56dad6afc1808111c986ebc8.json',
 '580a7456df23b703bf3599f0.json',
 '56e846b0c1808103936f74d4.json',
 '56e846c8c1808103936f7509.json',
 '587fae0adf23b703b94f83e8.json',
 '58ad4669df23b703bd5375bd.json',
 '58345a5bdf23b703bf9072f2.json',
 '579250fcc18081222835c029.json',
 '594bb068df23b703bd74cf61.json',
 '5760a732c1808103a5c97754.json',
 '5977798cdf23b703bdbb9ff0.json',
 '58172264df23b703bf50901d.json',
 'lexicon.pkl']

In [0]:
!pip3 install readability-lxml

In [0]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
pip install loader

In [0]:
import bs4
import time
import pickle
import logging
import codecs
import gensim
import unicodedata
import loader

from collections import Counter
from sklearn.model_selection import KFold

from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.api import CategorizedCorpusReader

from readability.readability import Unparseable
from readability.readability import Document as Paper
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize

from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.matutils import sparse2full
from gensim.corpora import Dictionary


log = logging.getLogger("readability.readability")
log.setLevel('WARNING')

DOC_PATTERN = r'(?!\.)[a-z_\s]+/[a-f0-9]+\.json'
PKL_PATTERN = r'(?!\.)[a-z_\s]+/[a-f0-9]+\.pickle'
CAT_PATTERN = r'([a-z_\s]+)/.*'

TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'li']


In [0]:
class JSONCorpusReader(CategorizedCorpusReader, CorpusReader):
    """ A corpus reader for JSON files to enable preprocessing."""

    def __init__(self, root, fileids=DOC_PATTERN, encoding='utf8', tags=TAGS, **kwargs):
        """Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining
        arguments are passed to the ``CorpusReader`` constructor."""
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN

        # Initialize the NLTK corpus reader objects
        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids, encoding)

        # Save the tags that we specifically want to extract.
        self.tags = tags

    def resolve(self, fileids, categories):
        """
        Returns a list of fileids or categories depending on what is passed
        to each internal corpus reader function. Implemented similarly to
        the NLTK ``CategorizedPlaintextCorpusReader``.
        """
        if fileids is not None and categories is not None:
            raise ValueError("Specify fileids or categories, not both")

        if categories is not None:
            return self.fileids(categories)
        return fileids

  
    def docs(self, fileids=None, categories=None):
        """
        Returns the complete text of an HTML document, closing the document
        after we are done reading it and yielding it in a memory safe fashion.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path, encoding in self.abspaths(fileids, include_encoding=True):
            with codecs.open(path, 'r', encoding=encoding) as f:
                yield f.read()

    def sizes(self, fileids=None, categories=None):
        """
        Returns a list of tuples, the fileid and size on disk of the file.
        This function is used to detect oddly large files in the corpus.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, getting every path and computing filesize
        for path in self.abspaths(fileids):
            yield os.path.getsize(path)
    
    def html(self, fileids=None, categories=None):
        """
        Returns the HTML content of each document, cleaning it using
        the readability-lxml library.
        """
        for doc in self.docs(fileids, categories):
            try:
                yield Paper(doc).summary()
            except Unparseable as e:
                print("Could not parse HTML: {}".format(e))
                continue

    import bs4
    TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'li']
    #....
    def paras(self, fileids=None, categories=None):
        """
        Uses BeautifulSoup to parse the paragraphs from the HTML.
        """
        for html in self.html(fileids, categories):
            soup = bs4.BeautifulSoup(html, 'lxml')
            for element in soup.find_all(self.tags):
                yield element.text
            soup.decompose()

    def sents(self, fileids=None, categories=None):
        """
        Uses the built in sentence tokenizer to extract sentences from the
        paragraphs. Note that this method uses BeautifulSoup to parse HTML.
        """
        for paragraph in self.paras(fileids, categories):
            for sentence in sent_tokenize(paragraph):
                yield sentence

    def words(self, fileids=None, categories=None):
        """
        Uses the built in word tokenizer to extract tokens from sentences.
        Note that this method uses BeautifulSoup to parse HTML content.
        """
        for sentence in self.sents(fileids, categories):
            for token in wordpunct_tokenize(sentence):
                yield token

    def tokenize(self, fileids=None, categories=None):
        """
        Segments, tokenizes, and tags a document in the corpus.
        """
        for paragraph in self.paras(fileids=fileids):
            yield [
                pos_tag(wordpunct_tokenize(sent))
                for sent in sent_tokenize(paragraph)
            ]

    def describe(self, fileids=None, categories=None):
        """
        Performs a single pass of the corpus and
        returns a dictionary with a variety of metrics
        concerning the state of the corpus.
        """
        started = time.time()

        # Structures to perform counting.
        counts  = nltk.FreqDist()
        tokens  = nltk.FreqDist()

        # Perform single pass over paragraphs, tokenize and count
        for para in self.paras(fileids, categories):
            counts['paras'] += 1

            for sent in sent_tokenize(para):
                counts['sents'] += 1

                for word in wordpunct_tokenize(sent):
                    counts['words'] += 1
                    tokens[word] += 1

        # Compute the number of files and categories in the corpus
        n_fileids = len(self.resolve(fileids, categories) or self.fileids())
        n_topics  = len(self.categories(self.resolve(fileids, categories)))

        # Return data structure with information
        return {
            'files':  n_fileids,
            'topics': n_topics,
            'paras':  counts['paras'],
            'sents':  counts['sents'],
            'words':  counts['words'],
            'vocab':  len(tokens),
            'lexdiv': float(counts['words']) / float(len(tokens)),
            'ppdoc':  float(counts['paras']) / float(n_fileids),
            'sppar':  float(counts['sents']) / float(counts['paras']),
            'secs':   time.time() - started,
        }

In [0]:
class Preprocessor(object):
    """
    The preprocessor wraps a corpus object (usually a `HTMLCorpusReader`)
    and manages the stateful tokenization and part of speech tagging into a
    directory that is stored in a format that can be read by the
    `HTMLPickledCorpusReader`. This format is more compact and necessarily
    removes a variety of fields from the document that are stored in the JSON
    representation dumped from the Mongo database. This format however is more
    easily accessed for common parsing activity.
    """

    def __init__(self, corpus, target=None, **kwargs):
        """
        The corpus is the `HTMLCorpusReader` to preprocess and pickle.
        The target is the directory on disk to output the pickled corpus to.
        """
        self.corpus = corpus
        self.target = target

    def fileids(self, fileids=None, categories=None):
        """
        Helper function access the fileids of the corpus
        """
        fileids = self.corpus.resolve(fileids, categories)
        if fileids:
            return fileids
        return self.corpus.fileids()

    def abspath(self, fileid):
        """
        Returns the absolute path to the target fileid from the corpus fileid.
        """
        # Find the directory, relative from the corpus root.
        parent = os.path.relpath(
            os.path.dirname(self.corpus.abspath(fileid)), self.corpus.root
        )

        # Compute the name parts to reconstruct
        basename  = os.path.basename(fileid)
        name, ext = os.path.splitext(basename)

        # Create the pickle file extension
        basename  = name + '.pickle'

        # Return the path to the file relative to the target.
        return os.path.normpath(os.path.join(self.target, parent, basename))

    def tokenize(self, fileid):
        """
        Segments, tokenizes, and tags a document in the corpus. Returns a
        generator of paragraphs, which are lists of sentences, which in turn
        are lists of part of speech tagged words.
        """
        for paragraph in self.corpus.paras(fileids=fileid):
            yield [
                pos_tag(wordpunct_tokenize(sent))
                for sent in sent_tokenize(paragraph)
            ]

    def process(self, fileid):
        """
        For a single file does the following preprocessing work:
            1. Checks the location on disk to make sure no errors occur.
            2. Gets all paragraphs for the given text.
            3. Segements the paragraphs with the sent_tokenizer
            4. Tokenizes the sentences with the wordpunct_tokenizer
            5. Tags the sentences using the default pos_tagger
            6. Writes the document as a pickle to the target location.
        This method is called multiple times from the transform runner.
        """
        # Compute the outpath to write the file to.
        target = self.abspath(fileid)
        parent = os.path.dirname(target)

        # Make sure the directory exists
        if not os.path.exists(parent):
            os.makedirs(parent)

        # Make sure that the parent is a directory and not a file
        if not os.path.isdir(parent):
            raise ValueError(
                "Please supply a directory to write preprocessed data to."
            )

        # Create a data structure for the pickle
        document = list(self.tokenize(fileid))

        # Open and serialize the pickle to disk
        with open(target, 'wb') as f:
            pickle.dump(document, f, pickle.HIGHEST_PROTOCOL)

        # Clean up the document
        del document

        # Return the target fileid
        return target

    def transform(self, fileids=None, categories=None):
        """
        Transform the wrapped corpus, writing out the segmented, tokenized,
        and part of speech tagged corpus as a pickle to the target directory.
        This method will also directly copy files that are in the corpus.root
        directory that are not matched by the corpus.fileids().
        """
        # Make the target directory if it doesn't already exist
        if not os.path.exists(self.target):
            os.makedirs(self.target)

        # Resolve the fileids to start processing and return the list of 
        # target file ids to pass to downstream transformers. 
        return [
            self.process(fileid)
            for fileid in self.fileids(fileids, categories)
        ]

In [0]:
url = '/content/gdrive/My Drive/Raw_Corpus_Sample'
corpus = JSONCorpusReader(url)

In [0]:
corpus.fileids()

['books/56d62a52c1808113ffb87f1d.json',
 'books/56d97be5c1808111c98555ef.json',
 'books/56dad6afc1808111c986ebc8.json',
 'books/56e846b0c1808103936f74d4.json',
 'books/56e846c8c1808103936f7509.json',
 'books/5760a732c1808103a5c97754.json',
 'books/579250fcc18081222835c029.json',
 'books/580a7456df23b703bf3599f0.json',
 'books/58172264df23b703bf50901d.json',
 'books/58345a5bdf23b703bf9072f2.json',
 'books/587fae0adf23b703b94f83e8.json',
 'books/58ad4669df23b703bd5375bd.json',
 'books/594bb068df23b703bd74cf61.json',
 'books/5977798cdf23b703bdbb9ff0.json',
 'cinema/5710062ec1808128a22d8d6e.json',
 'cinema/57111cbfc1808128a22fb04f.json',
 'cinema/57324feac180817cd239cd98.json',
 'cinema/5733a81cc1808103a3eff478.json',
 'cinema/578fb10ec1808122283046cf.json',
 'cinema/57ee9f2fdf23b703bff97467.json',
 'cinema/58121789df23b703bf45fb92.json',
 'cinema/5832a052df23b703bf8ca299.json',
 'cinema/58821e85df23b703b9544023.json',
 'cinema/58830743df23b703b9561bba.json',
 'cinema/58878186df23b703b95ea

In [0]:
preprocess = Preprocessor(corpus,r'/content/gdrive/My Drive/Raw_Corpus_Sample/data')

In [0]:
preprocess.transform()

['/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/56d62a52c1808113ffb87f1d.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/56d97be5c1808111c98555ef.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/56dad6afc1808111c986ebc8.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/56e846b0c1808103936f74d4.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/56e846c8c1808103936f7509.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/5760a732c1808103a5c97754.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/579250fcc18081222835c029.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/580a7456df23b703bf3599f0.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/58172264df23b703bf50901d.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/58345a5bdf23b703bf9072f2.pickle',
 '/content/gdrive/My Drive/Raw_Corpus_Sample/data/books/587fae0adf23b703b94f83e8.pickle',
 '/content

In [0]:
class PickledCorpusReader(CategorizedCorpusReader, CorpusReader):

    def __init__(self, root, fileids=PKL_PATTERN, **kwargs):
        """
        Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining arguments
        are passed to the ``CorpusReader`` constructor.
        """
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN

        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids)

    def resolve(self, fileids, categories):
        """
        Returns a list of fileids or categories depending on what is passed
        to each internal corpus reader function. This primarily bubbles up to
        the high level ``docs`` method, but is implemented here similar to
        the nltk ``CategorizedPlaintextCorpusReader``.
        """
        if fileids is not None and categories is not None:
            raise ValueError("Specify fileids or categories, not both")

        if categories is not None:
            return self.fileids(categories)
        return fileids

    def docs(self, fileids=None, categories=None):
        """
        Returns the document loaded from a pickled object for every file in
        the corpus. Similar to the BaleenCorpusReader, this uses a generator
        to acheive memory safe iteration.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path, enc, fileid in self.abspaths(fileids, True, True):
            with open(path, 'rb') as f:
                yield pickle.load(f)

    def paras(self, fileids=None, categories=None):
        """
        Returns a generator of paragraphs where each paragraph is a list of
        sentences, which is in turn a list of (token, tag) tuples.
        """
        for doc in self.docs(fileids, categories):
            for paragraph in doc:
                yield paragraph

    def sents(self, fileids=None, categories=None):
        """
        Returns a generator of sentences where each sentence is a list of
        (token, tag) tuples.
        """
        for paragraph in self.paras(fileids, categories):
            for sentence in paragraph:
                yield sentence

    def tagged(self, fileids=None, categories=None):
        for sent in self.sents(fileids, categories):
            for token in sent:
                yield token

    def words(self, fileids=None, categories=None):
        """
        Returns a generator of (token, tag) tuples.
        """
        for token in self.tagged(fileids, categories):
            yield token[0]

In [0]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, language='english'):
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords
      
    def normalize(self, document):
        return [
          self.lemmatize(token, tag).lower()
          for paragraph in document
          for sentence in paragraph
          for (token, tag) in sentence
          if not self.is_punct(token) and not self.is_stopword(token)
        ]
      
    def lemmatize(self, token, pos_tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag[0], wn.NOUN)
        
        return self.lemmatizer.lemmatize(token, tag)
    
    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        for document in documents:
            yield self.normalize(document)

In [0]:
class TextNormalizewithStemmer(BaseEstimator, TransformerMixin):
    
    def __init__(self, language='english'):
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.tokenizer = nltk.stem.SnowballStemmer('english')

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords
      
    def normalize(self, document):
        return [
          self.tokenize(token).lower()
          for paragraph in document
          for sentence in paragraph
          for (token, tag) in sentence
          if not self.is_punct(token) and not self.is_stopword(token)
        ]
      
    def tokenize(self, token):
      return self.tokenizer.stem(token)
    
    def fit(self, X, y=None):
      return self

    def transform(self, documents):
        for document in documents:
            yield self.normalize(document)

Removed Punctuations and Stopwords and Print the first 25 documents after applying Lemmatization

---



In [0]:
## Main Program Execution Starts here  ########       
# Use the path where you saved your input dataset in below statement
reader = PickledCorpusReader('/content/gdrive/My Drive/Raw_Corpus_Sample/data/')
docs = reader.docs()
normal = TextNormalizer()
normal.fit(docs)
docs = list(normal.transform(docs))
for x in range(25): 
    print (docs[x])

['n', 'n', 'n', 'r', 'n', 'r', 'nlast', 'week', 'ask', 'guaranteed', 'way', 'impress', 'book', 'lover', 'top', 'answer', 'friend', 'family', 'secret', 'admirer', 'need', 'nudge', 'right', 'direction', 'e', 'towards', 'bookstore', 'show', 'r', 'n', 'r', 'n1', 'give', 'book', 'lieu', 'bouquet', 'flower', 'u2014or', 'leave', 'sweet', 'note', 'book', 'reading', 'r', 'n', 'r', 'n2', 'flash', 'unexpired', 'library', 'card', 'r', 'n', 'r', 'n3', 'able', 'hold', 'interesting', 'conversation', 'book', 'author', 'like', 'like', 'share', 'opinion', 'thought', 'r', 'n', 'r', 'n4', 'casually', 'make', 'reference', 'book', 'mid', 'sentence', 'r', 'n', 'r', 'n5', 'gentle', 'book', 'toss', 'around', 'bend', 'cover', 'r', 'n', 'r', 'n6', 'read', 'favorite', 'book', 'someone', 'new', 'discuss', 'r', 'n', 'r', 'n7', 'tell', 'remind', 'specific', 'character', 'book', 'r', 'n', 'r', 'n8', 'listen', 'let', 'talk', 'character', 'situation', 'read', 'r', 'n', 'r', 'n9', 'lie', 'read', 'book', 'read', 'r', 'n'

Removed Punctuations and Stopwords and Print the first 25 documents after applying Stemming


In [0]:
## Main Program Execution Starts here  ########       
# Use the path where you saved your input dataset in below statement
from nltk.stem import PorterStemmer
reader_2 = PickledCorpusReader('/content/gdrive/My Drive/Raw_Corpus_Sample/data/')
docs_2 = reader_2.docs()
normal_2 = TextNormalizewithStemmer()
normal_2.fit(docs_2)
docs_2 = list(normal_2.transform(docs_2))
for x in range(25): 
    print (docs_2[x])

['n', 'n', 'n', 'r', 'n', 'r', 'nlast', 'week', 'ask', 'guarante', 'way', 'impress', 'book', 'lover', 'top', 'answer', 'friend', 'famili', 'secret', 'admir', 'need', 'nudg', 'right', 'direct', 'e', 'toward', 'bookstor', 'show', 'r', 'n', 'r', 'n1', 'give', 'book', 'lieu', 'bouquet', 'flower', 'u2014or', 'leav', 'sweet', 'note', 'book', 'read', 'r', 'n', 'r', 'n2', 'flash', 'unexpir', 'librari', 'card', 'r', 'n', 'r', 'n3', 'abl', 'hold', 'interest', 'convers', 'book', 'author', 'like', 'like', 'share', 'opinion', 'thought', 'r', 'n', 'r', 'n4', 'casual', 'make', 'refer', 'book', 'mid', 'sentenc', 'r', 'n', 'r', 'n5', 'gentl', 'book', 'toss', 'around', 'bend', 'cover', 'r', 'n', 'r', 'n6', 'read', 'favorit', 'book', 'someon', 'new', 'discuss', 'r', 'n', 'r', 'n7', 'tell', 'remind', 'specif', 'charact', 'book', 'r', 'n', 'r', 'n8', 'listen', 'let', 'talk', 'charact', 'situat', 'read', 'r', 'n', 'r', 'n9', 'lie', 'read', 'book', 'read', 'r', 'n', 'r', 'n10', 'alway', 'book', 'hand', 'r', 

Since Lemmatization provides a better output, I will use that output to apply TF-IDF vectorization using Gensim and Gensim Doc2Vec technique

TF-IDF Vectorization using Gensim on Lemmatized output

In [0]:
def gensim_tfidf_vectorize(corpus):
    
    import gensim
    import pickle

    corpus  = [list(doc) for doc in corpus]
    lexicon = gensim.corpora.Dictionary(corpus)
    tfidf   = gensim.models.TfidfModel(dictionary=lexicon, normalize=False)
    vector = [tfidf[lexicon.doc2bow(doc)] for doc in corpus]
    with open(r'/content/gdrive/My Drive/Raw_Corpus_Sample/data/TFIDF/Dictionary_TFIDF.pkl', 'wb') as f:
      pickle.dump(vector, f)

    lexicon.save_as_text(r'/content/gdrive/My Drive/Raw_Corpus_Sample/data/TFIDF/lexicon.txt')
    tfidf.save(r'/content/gdrive/My Drive/Raw_Corpus_Sample/data/TFIDF/tfidf.pkl')

    return vector

In [0]:
import warnings
warnings.filterwarnings("ignore")
for x in range(25): 
  print (gensim_tfidf_vectorize(docs)[x])

[(0, 2.678071905112638), (1, 5.0), (2, 2.415037499278844), (3, 1.4150374992788437), (4, 2.678071905112638), (5, 2.0), (6, 2.415037499278844), (7, 4.0), (8, 1.540568381362703), (9, 5.0), (10, 4.0), (11, 23.108525720440543), (12, 8.0), (13, 5.0), (14, 2.678071905112638), (15, 5.0), (16, 5.0), (17, 4.0), (18, 2.192645077942396), (19, 1.540568381362703), (20, 3.415037499278844), (21, 1.0), (22, 4.0), (23, 4.0), (24, 2.678071905112638), (25, 1.8300749985576878), (26, 4.385290155884792), (27, 5.0), (28, 5.0), (29, 1.299560281858908), (30, 1.8300749985576878), (31, 4.0), (32, 1.540568381362703), (33, 5.0), (34, 2.0), (35, 3.0), (36, 4.0), (37, 4.0), (38, 2.415037499278844), (39, 2.192645077942396), (40, 1.4150374992788437), (41, 2.192645077942396), (42, 1.6780719051126378), (43, 3.0), (44, 4.0), (45, 5.0), (46, 2.0342157153379135), (47, 2.415037499278844), (48, 2.678071905112638), (49, 0.8300749985576876), (50, 5.0), (51, 2.678071905112638), (52, 4.4308367926751036), (53, 5.0), (54, 5.0), (55

In [0]:
class GensimTFIDFVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, path=None):
        self.path = path
        self.id2word = None
        self.tfidf = None
        self.load()

    def load(self):
        if os.path.exists(self.path):
            self.id2word = gensim.corpora.Dictionary.load(self.path)
            
    def save(self):
        self.id2word.save(self.path)

    def fit(self, documents, labels=None):
        self.id2word = gensim.corpora.Dictionary(documents)
        self.tfidf   = gensim.models.TfidfModel(dictionary=self.id2word, normalize=False)
        
        self.save()
        return self

    def transform(self, documents):
        for document in documents:
            docvec = self.tfidf.id2word.doc2bow(document)
            yield sparse2full(docvec, len(self.tfidf.id2word))

In [0]:
vect = GensimTFIDFVectorizer('lexicon.pkl')
vect.fit(docs)

vect_docs = list(vect.transform(docs))

#Save the vectors
with open(r'/content/gdrive/My Drive/Raw_Corpus_Sample/data/TFIDF/Vectors_TFIDF.pkl', 'wb') as f:
      pickle.dump(vect_docs, f)

for x in range(25): 
    print (vect_docs[x])

[1. 1. 1. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[1. 0. 0. ... 0. 0. 0.]
[3. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 1. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 1. ... 0. 0. 0.]
[1. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 1. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 2. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 1. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


Gensim Doc2Vec technique

In [0]:
def gensim_doc2vec_vectorize(corpus):
    from gensim.models.doc2vec import TaggedDocument, Doc2Vec

    corpus = [list(doc) for doc in corpus]
    docs   = [
        TaggedDocument(words, ['d{}'.format(idx)])
        for idx, words in enumerate(corpus)
    ]
    model = Doc2Vec(docs, size=5, min_count=0)
    
    with open(r'/content/gdrive/My Drive/Raw_Corpus_Sample/data/TFIDF/Vector_Doc2Vec.pkl', 'wb') as f:
      pickle.dump(model.docvecs, f)
    
    return model.docvecs

In [0]:
warnings.filterwarnings("ignore")
for x in range(25): 
  print(gensim_doc2vec_vectorize(docs)[x])

[-0.98246014 -0.35554454  0.00384718  0.25819543 -0.88099444]
[-2.7283728  -0.768981   -0.32040274  0.49952057 -2.5171323 ]
[-1.1906805  -0.808912   -0.2525854   0.09857446 -1.5997121 ]
[-1.1485667  -0.33183068 -0.25608662  0.1242584  -1.1825016 ]
[-1.241529   -0.71789294 -0.3961687   0.11489222 -1.114939  ]
[-3.9847283  -0.29918873 -0.93835825  0.31328222 -2.5071704 ]
[-7.344425   -3.0299418  -1.4934543   0.91416866 -7.8565993 ]
[-2.1405575  -0.5395269  -0.11636375  0.41322178 -2.0548835 ]
[-13.180176   -2.9056509  -2.5689082   0.8616896 -10.8947935]
[-2.6639023  -0.57844716 -0.26533967  0.3917585  -2.3105536 ]
[-2.7977145  -0.5111275  -0.33808815  0.3794485  -2.6926978 ]
[-8.773184   -0.45823482 -1.3014908   0.636376   -6.786397  ]
[-11.538884   -7.7956243  -3.1250105  -0.695495   -7.282912 ]
[-0.81501997 -0.23115498 -0.04487555  0.28074828 -0.7413185 ]
[-1.8347868  -0.68603516 -0.47130066  0.1287715  -1.8004392 ]
[-1.5615814 -1.5194534 -0.4583579 -0.0699474 -1.4295275]
[-7.8546057 -